"""
Dependencies: numpy 1.17, pandas 0.25

userUserNetwork = W; Weighted adjacency matrix, probability user u respond to user v
treads = R
posts = P : user u, creation t, text x
documents = N
questionTendency = average; number of questions by total posts by user u in thread r for topic k
seeking (question) = S; QuestionTendency * log of 1+posts*length
disseminating (answer) = D; 1-Seeking
dictionary = X
topics = K
postTopics (theta) = [0,1]^N*K
topicWords = [0,1]^K*X
SIDR = phi; proportion of seeking by u on topic k by probability for user v responds to user u on topic k
DISR = psi; proportion of disseminating by u on topic k by probability for user u responds to user v on topic k
Benefit = B; utility obtained by user u for topic k; seeking*log of prob v to u on topic k
alpha = marginal benefit of teaching
smoothing = sigma
c_S, c_D = tightness parameters
step = lambda
t = threshold; error

Compute User-User Network
1-Smooth to ensure user responds to each post at most once
QuestionTendency = proportion of questions per topic per thread per weighted-average Q for u

Seeking and Dissemination
1:Extract forum topics: remove stopwords, urls, stem, lemmatize 
2:Infer if post is question or answer: first post and; other post or; has question mark or 5W1H or 1G
3:Compute S and D

Projected Gradient Descent
C_s = participation rate for seeking
C_d = participation rate for dissemination
alpha = learning step
"""

In [ ]:
thread = pd.read_json(r'/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread250-0',orient='records',typ='series',convert_dates=False)

In [ ]:
thread = {}
with open ('/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/designingcities-001/thread25-0', 'r') as f:
    thread = json.loads(f.read())

In [ ]:
json.load(open('/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/blendedlearning-001/thread25-0','r'))

In [518]:
import pandas as pd
import numpy as np
#import proxmin as px
import json, os

In [519]:
#Import Coursera course discussion forum files, concatenate threads
directory = r'/home/davidlemay/Documents/social_learning_network_analysis/Coursera MOOCs/courses/automata-002/'
threads = []
for f in os.listdir(directory):
    thread = json.load(open(os.path.join(directory,f),'r'))
    posts = pd.DataFrame(thread['posts'])
    posts['parent_id'] = posts.sort_values(['thread_id','order'])['user_id'].shift(+1)
    posts['id'].apply(pd.to_numeric)
    comments = pd.DataFrame(thread['comments'])
    threads.append(posts)
    posts.set_index('id',inplace=True)
    if comments.size > 0:
        comments['post_id'].apply(pd.to_numeric)
        comments.set_index('id',inplace=True)
        comments['parent_id'] = pd.Series(comments['post_id'].apply(lambda x: posts.loc[x, 'user_id']))
        threads.append(comments)
    else:
        continue
df = pd.concat(threads)

In [520]:
#ids = []
#for thread in threads:
#    if 'id' in thread.columns:
#        ids.append(thread['id'])
#group = list(pd.concat(ids))
#assert(len(group) - len(list(set(group))) == 0)

In [521]:
df.head()

,order,thread_id,user_id,post_time,edit_time,deleted,is_spam,original,stickied,approved,...,_viewer_can_vote,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,post_id,comment_text
id,,,,,,,,,,,,,,,,,,,,,
7,1.0,2.0,1556432.0,1.383556e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,NaN,NaN,NaN
10,2.0,2.0,688380.0,1.383557e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Hairong Zhu,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,1556432.0,NaN,NaN
12,3.0,2.0,5317955.0,1.383559e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Adrian de las Matas de la Fuente,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,688380.0,NaN,NaN
47,4.0,2.0,1106437.0,1.383589e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,5317955.0,NaN,NaN
60,5.0,2.0,1271.0,1.383594e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Instructor,Jeffrey Ullman,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,1106437.0,NaN,NaN


In [522]:
df[df['comment_text'].isna() == False].sort_values(['thread_id','order'])

,order,thread_id,user_id,post_time,edit_time,deleted,is_spam,original,stickied,approved,...,_viewer_can_vote,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,post_id,comment_text
id,,,,,,,,,,,,,,,,,,,,,
3,NaN,1.0,2376060.0,1.383568e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Rahul Kulkarni,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,3275.0,23.0,"Given a machine : generally - what it can do ,..."
5,NaN,1.0,3298925.0,1.383578e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Ahmed ALTAHER,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,3275.0,23.0,You will be familiar with machine and how to s...
9,NaN,1.0,5144921.0,1.383588e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Peter Koves,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,511954.0,22.0,"Yes, one of the most interesting things in thi..."
10,NaN,1.0,2376060.0,1.383591e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Rahul Kulkarni,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,898647.0,40.0,"Kudos Jim!<br /><br />For me , its tough to sa..."
16,NaN,1.0,5375487.0,1.383599e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Adnan Akhter,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,3275.0,23.0,basically we are studying languages and machin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,NaN,468.0,3921784.0,1.389408e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,Student,Derek Thompson,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,0.0,1691.0,"Yes. On the one hand, I take coursera courses..."
1294,NaN,468.0,0.0,1.389759e+09,NaN,0.0,0.0,NaN,NaN,NaN,...,True,False,NaN,NaN,NaN,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,1271.0,1696.0,No worries Dr. Ullman. As you say it's just a ...
1687,1.0,470.0,308182.0,1.389269e+09,-1.0,1.0,0.0,1.0,0.0,0.0,...,True,False,Student,Panagiotis Poursanidis,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,NaN,NaN,


In [523]:
df[df['comment_text'].isnull() == True]['comment_text'].size

1877

In [524]:
total_posts = 0
for x in threads:
    total_posts += len(x)
print(total_posts)

3269


In [525]:
#df.loc[2320,'user_id']

In [526]:
df[df['post_text'].str.contains('textbook') == True]

,order,thread_id,user_id,post_time,edit_time,deleted,is_spam,original,stickied,approved,...,_viewer_can_vote,_viewer_can_edit,_user_title,_user_full_name,_user_profile_link,link,_reporter_link,parent_id,post_id,comment_text
id,,,,,,,,,,,,,,,,,,,,,
601,3.0,134.0,4117766.0,1.384641e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Lauren Wood,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,973619.0,NaN,NaN
552,5.0,120.0,4881221.0,1.384497e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,973619.0,NaN,NaN
869,6.0,120.0,3853969.0,1.385438e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Paul Cardon,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,4881221.0,NaN,NaN
1641,19.0,387.0,1751306.0,1.388819e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Haseena V. AbdulRahaman,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,898647.0,NaN,NaN
337,1.0,88.0,1104131.0,1.384089e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Fernando Ramos,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,NaN,NaN,NaN
109,1.0,26.0,2169760.0,1.383628e+09,-1.0,0.0,0.0,1.0,0.0,0.0,...,True,False,Student,Anonymous,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,NaN,NaN,NaN
322,4.0,84.0,224778.0,1.384066e+09,-1.0,0.0,0.0,0.0,0.0,0.0,...,True,False,Student,Hobson Lane,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/forum/...,https://class.coursera.org/automata-002/help/flag,224778.0,NaN,NaN


In [527]:
df[df['post_id'].isnull() == True].index

Int64Index([   7,   10,   12,   47,   60,  787,  788,  815,    1,    2,
            ...
             541, 1514, 1519, 1111, 1112, 1116, 1119, 1683, 1684, 1685],
           dtype='int64', name='id', length=1968)

In [528]:
import re

def q_a(x):
    qWords = re.compile(r'[\w\W]*(who|what|where|when|why|how|\?)[\w\W]*')
    if re.search(qWords, x):
        return 'question'
    elif x != '':
        return 'answer'
    else:
        return None
        
df['q_a'] = df['post_text'].apply(lambda x: q_a(x) if type(x) == str else np.nan)

In [529]:
# EdX transforms
#df['_id'] = df['_id'].transform(lambda x: x['$oid'])
#df['parent_id'] = df['parent_id'].transform(lambda x: x['$oid'] if type(x) == dict else np.nan)

In [530]:
# EdX transform
#df.set_index('_id', inplace=True)
#df['parent_author_id'] = pd.Series(df['parent_id'].apply(lambda x: df.loc[x, 'author_id'] if type(x) == str else 0), dtype='Int64')
#df.reset_index(inplace=True)

In [531]:
df.to_csv('automata-forum.csv')

In [532]:
#df['post_text'] = df['post_text'].transform(lambda x: x if type(str) else "")

In [533]:
# _type : either CommentThread (i.e., initiation) or Comment (i.e., reply)
InitiationsReplies = pd.crosstab(df['user_id'], df['q_a'])
#InitiationsReplies.head()

In [534]:
# https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
#nltk.download('wordnet')

In [535]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [536]:
len(df['post_text'])

3269

In [537]:
#print(WordNetLemmatizer().lemmatize(gensim.utils.simple_preprocess(df['post_text'][1])[7], pos='v'))

In [538]:
#gensim.utils.simple_preprocess(df['post_text'][1])

In [539]:
processed_docs = df['post_text'].map(lambda x: preprocess(x) if type(x) == str else [])

In [540]:
processed_docs[:10]

id
7      [know, edit, text, introduct, automata, theori...
10                [know, assum, use, recent, edit, hand]
12                                                    []
47     [edit, actual, favorit, author, hopcroft, ullman]
60             [matter, edit, materi, mooc, cover, edit]
1                                                     []
22                                                    []
787    [accord, slide, lectur, exampl, give, confus, ...
788    [problem, function, sigma, rightarrow, domain,...
815    [sure, peopl, read, slashdot, miss, href, http...
Name: post_text, dtype: object

In [541]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [542]:
#dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [543]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [544]:
len(processed_docs)

3269

In [545]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [546]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.016*"cours" + 0.015*"proof" + 0.013*"think" + 0.012*"problem" + 0.009*"state" + 0.009*"automata" + 0.008*"final" + 0.008*"class" + 0.008*"time" + 0.007*"lectur"
Topic: 1 
Words: 0.023*"cours" + 0.010*"assign" + 0.010*"languag" + 0.010*"like" + 0.009*"class" + 0.009*"half" + 0.009*"program" + 0.009*"theori" + 0.008*"code" + 0.008*"lectur"
Topic: 2 
Words: 0.028*"languag" + 0.016*"cours" + 0.016*"string" + 0.011*"regular" + 0.009*"time" + 0.008*"question" + 0.007*"know" + 0.007*"thank" + 0.007*"like" + 0.006*"number"
Topic: 3 
Words: 0.020*"delta" + 0.018*"epsilon" + 0.012*"automata" + 0.012*"cours" + 0.011*"problem" + 0.011*"delta_" + 0.010*"work" + 0.009*"video" + 0.008*"half" + 0.008*"question"
Topic: 4 
Words: 0.043*"span" + 0.041*"state" + 0.023*"accept" + 0.016*"string" + 0.014*"final" + 0.012*"cours" + 0.011*"languag" + 0.010*"input" + 0.007*"start" + 0.007*"time"
Topic: 5 
Words: 0.040*"regular" + 0.033*"languag" + 0.029*"string" + 0.026*"state" + 0.016*"expres

In [547]:
topicScores = [lda_model.get_document_topics(x) for x in bow_corpus]

In [548]:
topics = pd.DataFrame([max(x, key=lambda y: y[1]) for x in topicScores])

In [549]:
df.insert(2, 'topics', topics[0])
#df.drop('topics', 1)
#topics.shape

In [550]:
len(df.post_text)

3269

In [551]:
df.columns

Index(['order', 'thread_id', 'topics', 'user_id', 'post_time', 'edit_time',
       'deleted', 'is_spam', 'original', 'stickied', 'approved', 'anonymous',
       'votes', 'post_text', 'user_agent', 'text_type', '_viewer_can_vote',
       '_viewer_can_edit', '_user_title', '_user_full_name',
       '_user_profile_link', 'link', '_reporter_link', 'parent_id', 'post_id',
       'comment_text', 'q_a'],
      dtype='object')

In [552]:
"""Create the adjacency matrix using _id for index and column labels"""
df1 = df.groupby(['post_id','user_id','parent_id'], sort=False, as_index=False)['user_id'].first()
userNetwork = pd.crosstab(df1.user_id,df1.parent_id)
idx = userNetwork.columns.union(userNetwork.index)
userNetwork = userNetwork.reindex(index=idx, columns=idx, fill_value=0)

In [553]:
df1[df1['post_id'] == 12]

,post_id,parent_id,user_id


In [554]:
userNetwork = pd.DataFrame(userNetwork.sort_index(axis=0).sort_index(axis=1))

In [555]:
#userNetwork.drop(0, axis=0, inplace=True)
#userNetwork.drop(0, axis=1, inplace=True)
#userNetwork.shape

In [556]:
#totalResponses = userNetwork.sum(axis=1)
totalPosts = userNetwork.sum(axis=0).transpose()
fracPosts = userNetwork/totalPosts

In [557]:
#w_UserNetwork = (fracPosts + (fracPosts.sum()/totalPosts.sum()))/(totalPosts + max(totalPosts))
#w_UserNetwork = user_network * total_responses / total_posts
w_UserNetwork = fracPosts

In [558]:
userNetwork.shape

(291, 291)

In [559]:
w_UserNetwork

,0.0,1270.0,1271.0,1652.0,3275.0,3907.0,6988.0,7927.0,23309.0,41377.0,...,5375487.0,5375777.0,5390874.0,5392276.0,5397453.0,5407819.0,5441187.0,5484838.0,5503583.0,5512672.0
0.0,0.270833,0.285714,0.127660,NaN,0.0,1.0,0.055556,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.2,NaN,NaN,NaN,0.0
1270.0,0.010417,0.000000,0.010638,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
1271.0,0.031250,0.000000,0.095745,NaN,0.0,0.0,0.055556,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.2,NaN,NaN,NaN,1.0
1652.0,0.010417,0.000000,0.000000,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
3275.0,0.000000,0.000000,0.010638,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5407819.0,0.000000,0.000000,0.010638,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.2,NaN,NaN,NaN,0.0
5441187.0,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
5484838.0,0.000000,0.000000,0.000000,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
5503583.0,0.010417,0.000000,0.000000,NaN,0.0,0.0,0.000000,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0


In [560]:
userTopics = pd.crosstab(df['user_id'], [df['q_a'], df['topics']])

In [561]:
userTopics.shape

(466, 20)

In [562]:
postTopics = userTopics.sum(0)/userTopics.sum(0).sum()

In [563]:
postTopics

q_a       topics
answer    0         0.244085
          1         0.031133
          2         0.025529
          3         0.016189
          4         0.028020
          5         0.031756
          6         0.012453
          7         0.020548
          8         0.019925
          9         0.016812
question  0         0.283313
          1         0.035492
          2         0.030511
          3         0.021793
          4         0.042341
          5         0.039228
          6         0.011208
          7         0.034247
          8         0.028643
          9         0.026775
dtype: float64

In [564]:
postingTendency = userTopics['question']*postTopics.sum()+userTopics['answer']*postTopics.sum()
questionTendency = (userTopics['question']*postTopics.sum())/postingTendency.sum()

In [565]:
questionTendency.shape

(466, 10)

In [566]:
#userTopics.columns = userTopics.columns.droplevel(0)
disseminating_full = (1-questionTendency * np.log(1+postingTendency))
seeking_full = (questionTendency * np.log(1+postingTendency))

In [567]:
idx = list(set(userNetwork.index.values).intersection(disseminating_full.index.values))

In [568]:
disseminating = disseminating_full.filter(items=idx,axis=0)
seeking = seeking_full.filter(items=idx,axis=0)
f_UserNetwork = w_UserNetwork.filter(items=idx,axis=0).filter(items=idx,axis=1)

In [569]:
f_UserNetwork.shape

(239, 239)

In [570]:
threshold = .01
alpha = .4
beta = 0.8
c_S = 1.25
c_D = 0.75
step = 0.1
rho = 1
N = f_UserNetwork.shape[0]
z1 = lambda1 = np.zeros(seeking.T.shape).astype('float64')
z2 = lambda2 = np.zeros(disseminating.shape).astype('float64')
W = W_hat = W_obs = f_UserNetwork.fillna(0).astype('float64').to_numpy()
W_opt = W_prime = np.zeros(W.shape)
D = disseminating.fillna(0).astype('float64').to_numpy()
S = seeking.fillna(0).astype('float64').to_numpy()

def SIDR(s,d,w):
    phi = np.zeros(s.shape)
    for u in range(0,s.shape[0]):
        for k in range(0,s.shape[1]):
            a = s[u,k]
            b = w.T[:,u].sum()*d[:,k].sum()
            phi[u,k] = np.divide(a,b, out=np.zeros_like(a), where=b!=0)
    return phi

def DISR(s,d,w):
    psi = np.zeros(d.shape)
    for u in range(d.shape[0]):
        for k in range(d.shape[1]):
            a = d[u,k]
            b = w[u,:].sum()*s[:,k].sum()
            psi[u,k] = np.divide(a,b, out=np.zeros_like(a), where=b!=0)
    return psi

    
#SIDR.replace(to_replace=0,value=1)
#DISR.replace(to_replace=0,value=1)
phi = c_S*SIDR(S,D,W)
psi = c_D*DISR(S,D,W)
P = np.divide(S,phi, out=np.zeros_like(S), where=phi!=0).T
Q = np.divide(D,psi, out=np.zeros_like(D), where=psi!=0)

In [571]:
np.seterr(all="raise")

def benefit(X):
    b = np.zeros(D.shape)
    for u in range(D.shape[0]):
        for k in range(D.shape[1]):
            b[u,k] = (S[u,k]*np.log(1+X[u].sum()*D[k].sum()) + alpha*D[u,k]*np.log(1+X[u].sum()*S[k].sum()))
    return b

def proj(X):
    """Projection step"""
    #return np.subtract(X,np.diag(np.diag(X)))/X.shape[0]/np.linalg.norm(X)
    return np.clip(np.subtract(X,np.diag(np.diag(X))),0,1)
    
def grad(X):
    """Proximal gradient step"""
    return (np.add((rho*np.dot(D,(np.add(np.subtract(np.dot(D.T,X),P),np.subtract(z1,lambda1))))),
            (rho*np.dot((np.subtract(np.subtract(np.dot(X,S),Q),np.add(z2,lambda2))),S.T))))

g = benefit(W).sum().sum()
g_hat = 1
i = 0

while (g - g_hat)/np.abs(g_hat) >= threshold:
    i += 1
    for u in range(W.shape[0]):
        for v in range(W.shape[1]):
            W_prime[u,v] = (
                (D[u].sum()*S[v].sum()/(1+(W[v].sum()*D.sum()).sum())
                +alpha*D[u].sum()*S[v].sum()/(1+(W[u].sum()*S.sum()).sum()))
                /N
            )
    W_hat = (W + step * W_prime)
    pj = proj(np.subtract(W,step*(np.subtract(W,np.add(W_hat,grad(W))))))
    W = pj
    #W = proj(grad(W_hat))
    g_hat = g
    g = benefit(W).sum().sum()
    z1 = np.maximum(np.add(np.add((-1*np.dot(D.T,W)),P),lambda1),0)
    z2 = np.maximum(np.add(np.subtract(np.dot(W,S),Q),lambda2),0)
    lambda1 -= np.subtract(np.add(np.dot(D.T,W),P),z1)
    lambda2 += np.subtract(np.subtract(np.dot(W,S),Q),z2)
    step *= beta

print("Iterations: "+str(i))
print("Observed learning benefit: "+str(benefit(W_obs).sum().sum()/N))
print("Optimized learning benefit: "+str(g_hat/N))

Iterations: 3
Observed learning benefit: 0.3533386488877507
Optimized learning benefit: 4.453196217235725


In [572]:
from numpy import linalg as la
import networkx as nx

In [573]:
nx.global_efficiency(nx.from_numpy_array(W))

0.9981540733448191

In [574]:
np.max(la.eig(W_obs)[0]).real

1.0

In [575]:
np.max(la.eig(W)[0]).real

77.22021475265412

In [576]:
np.savetxt('automata-optimized.csv',W,delimiter=',')